(jolly-hampton)=
# Stratified resampling

In [ ]:
%run ./geostatistical.ipynb

The {cite:t}`jolly_hampton_1990` algorithm for [estimating the mean and variance of stratified random transects](stratified-resampling-algo) is implemented using a [bootstrapping method](jolly-hampton-bootstrap). This enables the calculation of [confidence intervals](jolly-hampton-ci) for stratum-specific and and overall survey population estimates, as well as characterizing the overall variance via the coefficient of variation ($\textit{CV}$, {ref}`Eq. 2.21 <intext_eq_221_md>`).

## Import necessary modules

This algorithm is implemented via the `JollyHampton` class from the `survey` sub-package.

In [3]:
from echopop.survey import JollyHampton

The `JollyHampton` class can be initialized with two arguments:

- `model_parameters`: A dictionary that can be configured to include the following keys:
    - `num_replicates`: The number of bootstrap replicates.
    - `strata_transect_proportion`: The proportion of transects to sample per stratum. 
    - `transects_per_latitude`: The number of transects per degree latitude. However, this key is only necessary for virtual transect generation which will be discussed later. 
- `resample_seed`: An optional argument that sets the random seed for reproducible bootstrapping. If no seed is provided, then this defaults to being set to `None`. 

## Transects

For the transect data, we only need `strata_transect_proportion` and `num_replicates` for the `JollyHampton`-class: 

In [4]:
TRANSECT_JOLLYHAMPTON_PARAMETERS = {
    "strata_transect_proportion": 0.75,
    "num_replicates": 1000,
}

jh_transect = JollyHampton(TRANSECT_JOLLYHAMPTON_PARAMETERS)

The next step involves performing the bootstrap itself, which executes the built-in analysis pipeline. This is done by the `JollyHampton.stratified_bootstrap` method, which has three required inputs:

- `data_df`: Input `pandas.DataFrame` containing transect data.
- `stratify_by`: List of column names defining the strata (e.g., `["geostratum_inpfc"]`).
- `variable`: Name of the response variable column in `data_df` (e.g., `"biomass"`).

In [5]:
jh_transect.stratified_bootstrap(data_df=df_nasc_no_age1_prt, stratify_by=["geostratum_inpfc"], variable="biomass")

This method populates several attributes that can be accessed. The first is `transect_summary`, which details the summed biomass (kg), distance, area coverage, and areal/line biomass densities (kg nmi<sup>-2</sup>) for each transect (note that only the top ten rows are displayed below).

In [12]:
display(jh_transect.transect_summary.head(10))

biomass   distance        area  \
geostratum_inpfc transect_num                                        
1                1.0           4.010244e+06  34.809236  348.092365   
                 2.0           3.101135e+05  35.341287  353.532537   
                 3.0           0.000000e+00  50.660997  507.671065   
                 4.0           1.312977e+07  48.246504  483.030217   
                 5.0           0.000000e+00  48.119031  480.726902   
                 6.0           1.209238e+06  42.895494  428.615484   
                 7.0           1.541750e+06  45.527123  455.378065   
                 8.0           2.774609e+06  44.055394  440.435515   
                 9.0           3.618213e+06  40.192165  402.164809   
                 10.0          1.377956e+07  34.756911  347.653526   

                               biomass_areal_density  biomass_distance_density  
geostratum_inpfc transect_num                                                   
1                1.0                    11520.632646             115206.326459  
                 2.0                      877.185052               8774.820805  
                 3.0                        0.000000                  0.000000  
                 4.0                    27182.089235             272139.315187  
                 5.0                        0.000000                  0.000000  
                 6.0                     2821.266345              28190.337432  
                 7.0                     3385.648627              33864.430843  
                 8.0                     6299.694199              62980.008285  
                 9.0                     8996.842110              90022.850305  
                 10.0                   39635.892922             396455.200679

The next attribute is `strata_summary`, which provides the transect count (and resampling counts) total distance, area, biomass, and mean areal/line biomass densities for each stratum.

In [13]:
display(jh_transect.strata_summary.head(10))

,transect_counts,num_transects_to_sample,distance,area,biomass,biomass_distance_density,biomass_density
geostratum_inpfc,,,,,,,
1,10,8,424.604141,4247.300484,4.037350e+07,95085.028859,9505.684182
2,24,18,899.718778,10057.202226,4.257742e+08,473230.298614,42335.251530
3,13,10,504.277720,5773.969771,3.274201e+08,649285.338462,56706.242571
4,15,11,619.190882,7060.629519,4.466629e+08,721365.418732,63261.057510
5,15,11,666.979139,7068.645037,2.607504e+08,390942.338685,36888.312143
6,36,27,1392.286678,20673.571163,1.758493e+08,126302.502696,8505.994951


Next is `survey_summary`, which provides the overall total population estimate and density for each stratum and for the entire survey. These can be navigated between the two keys `"strata"` and `"survey"`.

In [38]:
jh_transect.survey_summary["strata"]

biomass                                            \
geostratum_inpfc             1             2             3             4   
0                 4.037350e+07  4.257742e+08  3.274201e+08  4.466629e+08   

                                             biomass_density               \
geostratum_inpfc             5             6               1            2   
0                 2.607504e+08  1.758493e+08     9505.684182  42335.25153   

                                                                        
geostratum_inpfc             3            4             5            6  
0                 56706.242571  63261.05751  36888.312143  8505.994951

In [39]:
jh_transect.survey_summary["survey"]

,biomass,biomass_density,cv
0,1.677048e+09,30553.755539,0.109971


The last attribute to note is `bootstrap_replicates`, which is a DataFrame comprising the computed estimators (e.g., total coverage area, biomass, weighted biomass density) for each of the bootstrap replicates. This can be used to gain insights from the distributions of different metrics within each strata and for the entire survey across replicates.

In [40]:
display(jh_transect.bootstrap_replicates.head(10))

area                                         \
geostratum_inpfc            1            2            3            4   
replicate                                                              
0                 3491.603138  7509.820151  4511.928114  5513.006534   
1                 3362.105458  7484.795160  4437.237772  5335.221508   
2                 3335.654784  7954.843829  4213.283876  4795.224655   
3                 3545.675582  7623.011593  4658.918907  4770.894617   
4                 3404.700160  7536.521289  4522.591358  4976.292893   
5                 3545.675582  7361.792693  4681.977201  5329.516331   
6                 3453.332432  7778.876291  4661.664549  5358.256039   
7                 3459.211443  7467.118724  4480.570122  5018.232907   
8                 3363.306935  7710.787861  4452.249166  5246.909016   
9                 3418.481217  7684.859244  4684.746919  5377.169006   

                                                  biomass                \
geostratum_inpfc            5             6             1             2   
replicate                                                                 
0                 5457.315275  14318.621027  4.555646e+07  4.346331e+08   
1                 5092.035493  15363.064304  2.953189e+07  3.314311e+08   
2                 5210.857907  17234.327973  3.254311e+07  4.857275e+08   
3                 5377.238005  16149.719545  4.506642e+07  3.828920e+08   
4                 4742.204911  13191.110861  4.247584e+07  4.267015e+08   
5                 5015.109211  13626.957423  4.506642e+07  3.431288e+08   
6                 5228.580027  14272.287058  4.661097e+07  4.222089e+08   
7                 5338.222629  17603.312432  2.977416e+07  3.009117e+08   
8                 5202.153736  16506.412792  4.702814e+07  4.543071e+08   
9                 5397.603790  15359.356385  4.545407e+07  4.379739e+08   

                                              ...  \
geostratum_inpfc             3             4  ...   
replicate                                     ...   
0                 3.410848e+08  5.305976e+08  ...   
1                 3.443247e+08  4.483489e+08  ...   
2                 2.918134e+08  3.245640e+08  ...   
3                 3.510499e+08  3.718640e+08  ...   
4                 3.251184e+08  5.221232e+08  ...   
5                 3.219473e+08  4.529638e+08  ...   
6                 3.239509e+08  3.985688e+08  ...   
7                 3.543826e+08  4.984857e+08  ...   
8                 2.978632e+08  4.019570e+08  ...   
9                 3.341958e+08  4.049182e+08  ...   

                 distance_weighted_biomass_density                \
geostratum_inpfc                                 3             4   
replicate                                                          
0                                     2.828432e+07  3.448754e+07   
1                                     2.876824e+07  2.998217e+07   
2                                     2.242866e+07  2.210942e+07   
3                                     2.898080e+07  2.488468e+07   
4                                     2.706904e+07  3.381357e+07   
5                                     2.698873e+07  2.966433e+07   
6                                     2.712935e+07  2.631528e+07   
7                                     2.924806e+07  3.228357e+07   
8                                     2.507716e+07  2.611321e+07   
9                                     2.782286e+07  2.646311e+07   

                                             distance_weighted_variance  \
geostratum_inpfc             5             6                          1   
replicate                                                                 
0                 1.998586e+07  7.926901e+06               2.551602e+12   
1                 2.131988e+07  2.120924e+06               9.404633e+11   
2                 2.090768e+07  7.328970e+06               1.171072e+12   
3                 2.314116e+07  6.932840e+06               2.483088e+12   
4           

These bootstrap replicates can then be processed into summary statistics computed for each stratum as well as the entire survey. This uses the `JollyHampton.summarize` method, which as the arguments:

- `ci_percentile`: Confidence level for estimating the uncertainty interval. This defaults to `0.95`.
- `ci_method`: The method used for computing the confidence interval ($\textit{CI}$). Currently, valid entries for `ci_method` are:
    - `"student_jackknife"`: This is the default method that computes the studentized $\textit{CI}$ using jackknife (or leave-one-out) resampling.
    - `"bc"`: Bias-corrected $\textit{CI}$ that adjusts for bias in the bootstrap distribution using the empirical cumulative distribution function.
    - `"bca"`: Bias-corrected and accelerated $\textit{CI}$ that not only accounts for bias in the bootstrap sample but also corrects for skewness using finite-sample jackknife resampling.
    - `"empirical"`: Sometimes known as the "delta method", this selection uses the distribution of bootstrapped deviations between the replicate means and population statistic.
    - `"normal"`: This assumes that the bootstrap replicates are normally distributed.
    - `"percentile"`: The $\textit{CI}$ are constructed directly from the bootstrap distribution quantiles.
    - `"student_standard"`: This assumes that the bootstrap replicates are approximately $t$-distributed.

In [41]:
jh_transect.summarize()

biomass                                            \
metric           low          mean          high          bias   
1       2.820906e+07  4.040993e+07  5.010102e+07  3.643418e+04   
2       3.070271e+08  4.225634e+08  5.103502e+08 -3.210779e+06   
3       2.437218e+08  3.256368e+08  3.793792e+08 -1.783342e+06   
4       3.242482e+08  4.452046e+08  5.418683e+08 -1.458327e+06   
5       1.935299e+08  2.630395e+08  3.194195e+08  2.289147e+06   
6       8.319200e+07  1.750265e+08  2.291056e+08 -8.227529e+05   
survey  1.462387e+09  1.671881e+09  1.865804e+09 -5.166822e+06   

       biomass_density                                                cv  \
metric             low          mean          high        bias       low   
1          6641.644748   9514.262378  11795.966714    8.578196       NaN   
2         30528.078222  42015.999774  50744.749903 -319.251756       NaN   
3         42210.443573  56397.383683  65705.088947 -308.858888       NaN   
4         45923.418822  63054.514082  76745.039140 -206.543428       NaN   
5         27378.645785  37212.157417  45188.219506  323.845274       NaN   
6          4024.075188   8466.197624  11082.052377  -39.797327       NaN   
survey    26646.356142  30463.567847  33997.061607  -90.187691  0.123976   

                                      
metric      mean      high      bias  
1            NaN       NaN       NaN  
2            NaN       NaN       NaN  
3            NaN       NaN       NaN  
4            NaN       NaN       NaN  
5            NaN       NaN       NaN  
6            NaN       NaN       NaN  
survey  0.130357  0.141477  0.020386

This yields a `pandas.DataFrame` indexed by each stratum and overall survey, with columns organized by their respective metrics. In this case, we have `"biomass"`, `"biomass_density"`, and `"cv"`. The columns `"low"`, `"mean"`, and `"high"` correspond to the lower bound of the $\textit{CI}$, the distribution mean, and the upper bound of the $\textit{CI}$, respectively. The `"bias"` represents the deviation between the bootstrapped means and the original estimates. The metric `"cv"` is only calculated for the entire survey, so there are no valid values to report for each stratum (i.e., `NaN`).

## Kriged mesh

We can also run this analysis for the kriged mesh estimates, although we have to incorporate a few modifications to make the gridded points compatible with the expected transect sampling design. So we now initialize the `JollyHampton`-class object with the dictionary key `"transects_per_latitude"`, which defines the number of virtual transects that will be generated per degree latitude. 

In [42]:
KRIGED_JOLLYHAMPTON_PARAMETERS = {
    "transects_per_latitude": 5,
    "strata_transect_proportion": 0.75,
    "num_replicates": 1000,
}

jh_kriged = JollyHampton(KRIGED_JOLLYHAMPTON_PARAMETERS)

Before using the `JollyHampton.stratified_bootstrap` method, the `JollyHampton.create_virtual_transects` method is required. This method creates virtual transects from the gridded data that are then subsequently assigned to individual strata. This has four arguments:
- `data_df`: A `pandas.DataFrame` containing gridded kriged data.
- `geostrata_df`: A `pandas.DataFrame` containing geographical stratum boundaries and definitions (e.g., `"geostratum_inpfc"`).
- `stratify_by`: A list of column names in `data_df` to stratify by (e.g., `"geostratum_inpfc"`).
- `variable`: Name of the response variable column in `data_df` (e.g., `"biomass"`).

This returns a `pandas.DataFrame` that can then be fed into the other `JollyHampton`-class methods as if they were standard transect data.

In [ ]:
kriged_transects = jh_kriged.create_virtual_transects(
    data_df=unextrapolated_results,
    geostrata_df=df_dict_geostrata["inpfc"],
    stratify_by=["geostratum_inpfc"],
    variable="biomass",
)

In [45]:
display(kriged_transects.head(10))

,transect_num,latitude,transect_distance,transect_area,biomass,geostratum_inpfc
0,1,34.6,48.675265,292.051592,1.278193e+06,1
1,2,34.8,54.211796,650.541558,1.061705e+06,1
2,3,35.0,54.655882,655.870584,1.071039e+07,1
3,4,35.2,56.585071,679.020851,6.468483e+05,1
4,5,35.4,51.338622,616.063464,1.791419e+06,1
5,6,35.6,57.280884,687.370604,3.334136e+06,1
6,7,35.8,51.768742,621.224904,3.610589e+06,1
7,8,36.0,48.788137,585.457646,1.494046e+07,2
8,9,36.2,47.058672,564.704058,8.543800e+06,2
9,10,36.4,44.176795,530.121544,3.189396e+06,2


So now we can take these virtual transects to compute the bootstrap replicates and subsequent statistics.

In [46]:
# Generate replicates
jh_kriged.stratified_bootstrap(data_df=kriged_transects, stratify_by=["geostratum_inpfc"], variable="biomass")

# Summarize
jh_kriged.summarize()

biomass                                            \
metric           low          mean          high          bias   
1       1.078635e+07  2.232470e+07  2.905677e+07 -1.085791e+05   
2       3.082708e+08  3.940729e+08  4.648138e+08  2.132049e+05   
3       2.565377e+08  3.439625e+08  4.101752e+08 -9.524314e+05   
4       3.903927e+08  4.926153e+08  5.581010e+08  2.772988e+06   
5       1.880855e+08  2.663654e+08  3.106342e+08  1.098833e+06   
6       1.191697e+08  1.966672e+08  2.516261e+08  8.902240e+04   
survey  1.517318e+09  1.716008e+09  1.876902e+09  2.931529e+06   

       biomass_density                                                cv  \
metric             low          mean          high        bias       low   
1          2540.437518   5257.987864   6843.547876  -25.572912       NaN   
2         23037.811113  29450.005131  34736.640264   15.933312       NaN   
3         36675.415948  49173.923503  58639.899534 -136.162483       NaN   
4         49920.542282  62992.023841  71365.848143  354.589325       NaN   
5         20822.068401  29488.068006  34388.863662  121.646641       NaN   
6          3320.180188   5479.333932   7010.538926    2.480248       NaN   
survey    19611.808670  22179.938517  24259.542433   40.236983  0.128841   

                                      
metric      mean      high      bias  
1            NaN       NaN       NaN  
2            NaN       NaN       NaN  
3            NaN       NaN       NaN  
4            NaN       NaN       NaN  
5            NaN       NaN       NaN  
6            NaN       NaN       NaN  
survey  0.134447  0.141746  0.018865